In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import joblib

In [4]:
df = pd.read_csv('/Users/laraturunc/Desktop/churn data/Telco-Customer-Churn-Dataset.csv')


df.drop('customerID', axis=1, inplace=True)

In [5]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].fillna(df['TotalCharges'].median(), inplace=True)


df.fillna(method='ffill', inplace=True)

In [26]:
df['Contract_Tenure'] = df['Contract'].astype(str) + '_' + df['tenure'].astype(str) + '_' + df['PaymentMethod'].astype(str)


services = ['PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']
df['ServicesUsed'] = df[services].sum(axis=1)

In [27]:
le = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = le.fit_transform(df[column])

In [28]:
scaler = StandardScaler()
df[df.columns] = scaler.fit_transform(df)

In [29]:
X = df.drop('Churn', axis=1) 
y = df['Churn']

le_churn = LabelEncoder()
y = le_churn.fit_transform(df['Churn'])


smote = SMOTE()
X, y = smote.fit_resample(X, y)

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [20]:
#MACİ KIZARTACAKTI DURDURDUM

# from sklearn.model_selection import train_test_split, GridSearchCV

# grid = GridSearchCV(lgbm, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
# grid.fit(X_train, y_train)


# best_lgbm = grid.best_estimator_


# importances = best_lgbm.feature_importances_
# # Selecting features which have non-zero importance
# selected_features = X_train.columns[(importances > 0)]
# X_train_selected = X_train[selected_features]
# X_test_selected = X_test[selected_features]

# best_lgbm.fit(X_train_selected, y_train)


# predictions = best_lgbm.predict(X_test_selected)
# accuracy = accuracy_score(y_test, predictions)
# precision = precision_score(y_test, predictions)
# recall = recall_score(y_test, predictions)
# f1 = f1_score(y_test, predictions)


# print(f'Best Parameters: {grid.best_params_}')
# print(f'Accuracy: {accuracy}')
# print(f'Precision: {precision}')
# print(f'Recall: {recall}')
# print(f'F1 Score: {f1}')


# joblib.dump(best_lgbm, 'telco_customer_churn_model_improved.pkl')

[LightGBM] [Info] Number of positive: 2915, number of negative: 2880
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003645 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1717
[LightGBM] [Info] Number of data points in the train set: 5795, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503020 -> initscore=0.012080
[LightGBM] [Info] Start training from score 0.012080
[LightGBM] [Info] Number of positive: 2914, number of negative: 2880
[LightGBM] [Info] Number of positive: 2915, number of negative: 2880
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015589 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1721
[LightGBM] [Info] Number of data points in the train set: 5794, number of used features: 19
[LightGBM] [Info] Num

Exception ignored on calling ctypes callback function: <function _log_callback at 0x120c68e50>
Traceback (most recent call last):
  File "/Users/laraturunc/opt/anaconda3/envs/env_123/lib/python3.8/site-packages/lightgbm/basic.py", line 203, in _log_callback
    def _log_callback(msg: bytes) -> None:
KeyboardInterrupt: 


KeyboardInterrupt: 

In [21]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

param_dist = {
    'num_leaves': sp_randint(20, 50),
    'reg_alpha': [0.1, 0.5],
    'reg_lambda': [0.1, 0.5],
    'min_child_weight': [0.001, 0.01],
    'learning_rate': [0.1, 0.01]
}


lgbm = LGBMClassifier(n_estimators=100)


random_search = RandomizedSearchCV(lgbm, param_distributions=param_dist, n_iter=10, cv=3, scoring='accuracy', n_jobs=-1, random_state=42)
random_search.fit(X_train, y_train)


best_lgbm = random_search.best_estimator_




[LightGBM] [Info] Number of positive: 2429, number of negative: 2400
[LightGBM] [Info] Number of positive: 2429, number of negative: 2400
[LightGBM] [Info] Number of positive: 2429, number of negative: 2400
[LightGBM] [Info] Number of positive: 2429, number of negative: 2400
[LightGBM] [Info] Number of positive: 2428, number of negative: 2400
[LightGBM] [Info] Number of positive: 2429, number of negative: 2400
[LightGBM] [Info] Number of positive: 2428, number of negative: 2400
[LightGBM] [Info] Number of positive: 2428, number of negative: 2400
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004276 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1573
[LightGBM] [Info] Number of data points in the train set: 4829, number of used features: 19
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.503003 -> initscore=0.012011
[LightGBM] [Info]

In [24]:
predictions = best_lgbm.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)


print(f'Best Parameters: {random_search.best_params_}')
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')


joblib.dump(best_lgbm, 'telco_customer_churn_model_improved_NOT.pkl')


Best Parameters: {'learning_rate': 0.1, 'min_child_weight': 0.01, 'num_leaves': 27, 'reg_alpha': 0.5, 'reg_lambda': 0.1}
Accuracy: 0.8573268921095009
Precision: 0.8505154639175257
Recall: 0.8621815806662312
F1 Score: 0.8563087901394744


['telco_customer_churn_model_improved_NOT.pkl']